In [288]:
import glob
import os
import pandas as pd
import seaborn as sns
import re
import numpy as np

df = pd.read_csv ('forms.csv')          # 
hausa = df[(df.Language_ID == 4)]


/var/folders/s_/frmpdrds50v8kvsf17kwk1180000gn/T/ipykernel_41144/904658155.py:9: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv ('forms.csv')          #


In [294]:
hausa = (hausa.loc[(hausa['BorrowedScore'] == 0) & (hausa['Analyzability'] == "unanalyzable")])
cplx_onset = ['ts','gw','ƙw','ƙy','sh','kw']
coda = 'r̃mnrl'
reduced_words = hausa[~hausa['Form'].str.contains('\\d|,|`|\'|\$|\ \(|\)',regex=True)].drop_duplicates(subset = 'Form')
reduced_words.index = np.arange(len(reduced_words))
len(reduced_words)
#reduced_words

665

In [296]:
reduced_words = reduced_words[~reduced_words['Form'].str.contains ('|'.join(cplx_onset))]
words = reduced_words['Form']


In [309]:
h_tone = "áéíóú"
l_tone = "àèìòù"
f_tone = "âêîôû"
long_vowel = 'aeiou'
v = h_tone + l_tone + f_tone

class Autorep:   
    def __init__(self, word):
        self.word = word
        self.tone = ''
        self.mldy = ''
        self.assoc = []
        
        # Initialize tone outside the loop to accumulate tones
        tone = ''
        for i,seg in enumerate(word):
            #if seg in h_tone and word[i+1] in long_vowel:
            #    tone += "H:"
            #elif seg in l_tone and word[i+1] in long_vowel:
            #    tone += "L:"
            if seg in f_tone:
                tone += "F"
            elif seg in h_tone:
                tone += "H" 
            elif seg in l_tone:
                tone += "L"      
        self.tone = tone 
        self.mldy = re.sub(r"(.)\1+", r"\1", tone)
          
        if len(self.tone) == len(self.mldy):
            for i in range(len(self.mldy)):
                if self.tone[i] == self.mldy[i]:
                    self.assoc.append((i+1, i+1))

        elif len(self.tone) != len(self.mldy):
            j = 0
            for i in range(len(self.mldy)):
                while j < len(self.tone): 
                    if self.mldy[i] == self.tone[j]:
                        self.assoc.append((i+1,j+1))
                        j += 1
                    elif self.mldy[i] != self.tone[j]:
                        break
        print(self.word, self.tone, self.mldy, self.assoc)

example_word = "káakásàsâ"
auto_rep = Autorep(example_word)


káakásàsâ HHLF HLF [(1, 1), (1, 2), (2, 3), (3, 4)]


Some questions:
- Is mora tier needed in the representation (i.e. does length difference matter)   
  - heavy syllables
  - complex onset vs codas
- How to handle F -> HL
